
# LeanTrader — XAUUSD Walk-Forward Validation

This notebook performs **walk-forward validation** on XAUUSD strategy parameters.

- Splits historical data into **rolling training windows** and **test windows**.
- For each split: runs a parameter sweep (ADX, ATR stop, R multiple, FVG lookback).
- Selects best params from training → applies to test → records performance.
- Reports average test Sharpe, MaxDD, and consistency.

This prevents overfitting and ensures robustness across time periods & sessions.


In [ ]:

import os, yaml, itertools
import pandas as pd
from pathlib import Path

from leantrader.data.feeds import get_ohlc_csv, resample_frames
from leantrader.features.ta import rsi, adx, fvg_score
from leantrader.dsl.compiler import compile_strategy, load_strategy
from leantrader.backtest.engine import backtest
from leantrader.backtest.metrics import sharpe, max_drawdown

REPO_ROOT = Path.cwd().resolve()
DATA_DIR = REPO_ROOT / "data" / "ohlc"
PAIR = "XAUUSD"
m15_path = DATA_DIR / f"{PAIR}_M15.csv"
assert m15_path.exists(), f"Missing {m15_path}"
df_m15 = get_ohlc_csv(str(m15_path))
frames = resample_frames(df_m15)

# Engineering with param
def engineer_param(df: pd.DataFrame, fvg_lb: int = 3) -> pd.DataFrame:
    out = df.copy()
    out["rsi_14"] = rsi(out["close"], 14)
    out["adx_14"] = adx(out, 14)
    out["fvg_score"] = fvg_score(out, fvg_lb)
    return out

spec_path = REPO_ROOT / "src" / "leantrader" / "dsl" / "examples" / "xauusd_master.yaml"
base_spec = load_strategy(str(spec_path))

def spec_with_params(spec, adx_thr: int, atr_stop: float, r_mult: float):
    import copy
    s = copy.deepcopy(spec)
    for sig in s.signals:
        for cond in sig.entry:
            if cond.feature.startswith("adx_14"):
                cond.feature = f"adx_14 > {adx_thr}"
        sig.stop = f"atr_14 * {atr_stop}"
        sig.take = f"R:{r_mult}"
    return s

param_space = {
    "adx_thr": [18, 20, 22],
    "atr_stop": [1.4, 1.6],
    "r_mult": [2.0, 2.2],
    "fvg_lb": [2, 3]
}


In [ ]:

import numpy as np

# Build walk-forward splits: 70% train, 30% test with rolling
times = frames["M15"].index
n = len(times)
window = int(n * 0.7)
step = int(n * 0.1)

splits = []
for start in range(0, n-window, step):
    train_idx = times[start:start+window]
    test_idx = times[start+window:start+window+step]
    if len(test_idx)==0: break
    splits.append((train_idx, test_idx))

print("Total splits:", len(splits))


In [ ]:

results = []
for i,(train_idx, test_idx) in enumerate(splits):
    best_train = None
    best_score = -999
    for adx_thr in param_space["adx_thr"]:
        for atr_stop in param_space["atr_stop"]:
            for r_mult in param_space["r_mult"]:
                for fvg_lb in param_space["fvg_lb"]:
                    eng = {k: engineer_param(v.loc[train_idx], fvg_lb=fvg_lb) for k,v in frames.items()}
                    spec = spec_with_params(base_spec, adx_thr, atr_stop, r_mult)
                    strat = compile_strategy(spec)
                    sigs = strat(eng)
                    m15 = eng["M15"]
                    sigs["price"] = m15["close"].reindex(sigs.index, method="ffill")
                    eq = backtest(m15, sigs, risk_cfg=None)
                    if len(eq)==0: continue
                    score = float(sharpe(eq)) - abs(float(max_drawdown(eq)))
                    if score > best_score:
                        best_score = score
                        best_train = (adx_thr, atr_stop, r_mult, fvg_lb)
    # test with best params
    if best_train:
        adx_thr, atr_stop, r_mult, fvg_lb = best_train
        eng_test = {k: engineer_param(v.loc[test_idx], fvg_lb=fvg_lb) for k,v in frames.items()}
        spec = spec_with_params(base_spec, adx_thr, atr_stop, r_mult)
        strat = compile_strategy(spec)
        sigs = strat(eng_test)
        m15 = eng_test["M15"]
        sigs["price"] = m15["close"].reindex(sigs.index, method="ffill")
        eq = backtest(m15, sigs, risk_cfg=None)
        if len(eq)>0:
            results.append({
                "split": i,
                "train_params": best_train,
                "test_sharpe": float(sharpe(eq)),
                "test_maxdd": float(max_drawdown(eq))
            })


In [ ]:

import pandas as pd
res_df = pd.DataFrame(results)
res_df


In [ ]:

# Aggregate performance across all test windows
agg = res_df[["test_sharpe","test_maxdd"]].agg(["mean","std","min","max"])
agg



## Next
- If test Sharpe is stable and MaxDD reasonable, deploy these params live.
- If variance is high, expand parameter space or refine DSL logic.
